# Notebook 34: Peer-to-Peer Memory Transfer## Phase 6: Streams & Concurrency**Learning Objectives:**- Understand P2P- Learn GPU Direct- Master inter-GPU transfer- Apply concepts in practical scenarios- Measure and analyze performance

## Concept: Peer-to-Peer Memory Transfer**Topics Covered:**- P2P- GPU Direct- inter-GPU transfer**Key Concepts:**This notebook covers P2P in the context of Phase 6: Streams & Concurrency.

## Example 1: Basic Peer-to-Peer Memory Transfer

In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <math.h>

#define CUDA_CHECK(call) \
    do { \
        cudaError_t err = call; \
        if (err != cudaSuccess) { \
            fprintf(stderr, "CUDA error at %s:%d: %s\n", \
                    __FILE__, __LINE__, cudaGetErrorString(err)); \
            exit(EXIT_FAILURE); \
        } \
    } while(0)


int main() {
    printf("=== Peer-to-Peer Transfer ===\n\n");

    int deviceCount;
    CUDA_CHECK(cudaGetDeviceCount(&deviceCount));
    printf("GPUs available: %d\n\n", deviceCount);

    if (deviceCount < 2) {
        printf("P2P requires 2+ GPUs\n");
        return 0;
    }

    int canAccessPeer;
    CUDA_CHECK(cudaDeviceCanAccessPeer(&canAccessPeer, 0, 1));
    printf("GPU 0 can access GPU 1: %s\n", canAccessPeer ? "Yes" : "No");

    if (canAccessPeer) {
        CUDA_CHECK(cudaSetDevice(0));
        CUDA_CHECK(cudaDeviceEnablePeerAccess(1, 0));

        const int N = 1 << 20;
        size_t bytes = N * sizeof(float);

        float *d_data0, *d_data1;
        CUDA_CHECK(cudaSetDevice(0));
        CUDA_CHECK(cudaMalloc(&d_data0, bytes));

        CUDA_CHECK(cudaSetDevice(1));
        CUDA_CHECK(cudaMalloc(&d_data1, bytes));

        cudaEvent_t start, stop;
        CUDA_CHECK(cudaEventCreate(&start));
        CUDA_CHECK(cudaEventCreate(&stop));

        CUDA_CHECK(cudaSetDevice(0));
        CUDA_CHECK(cudaEventRecord(start));
        CUDA_CHECK(cudaMemcpyPeer(d_data1, 1, d_data0, 0, bytes));
        CUDA_CHECK(cudaEventRecord(stop));
        CUDA_CHECK(cudaEventSynchronize(stop));

        float ms;
        CUDA_CHECK(cudaEventElapsedTime(&ms, start, stop));
        printf("P2P transfer: %.3f ms, BW: %.2f GB/s\n",
               ms, bytes / (ms * 1e6));

        CUDA_CHECK(cudaSetDevice(0));
        cudaFree(d_data0);
        CUDA_CHECK(cudaSetDevice(1));
        cudaFree(d_data1);

        cudaEventDestroy(start); cudaEventDestroy(stop);
    }

    return 0;
}


## Practical ExerciseComplete the following exercises to practice the concepts learned.

In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>

#define CUDA_CHECK(call) \
    do { \
        cudaError_t err = call; \
        if (err != cudaSuccess) { \
            fprintf(stderr, "CUDA error at %s:%d: %s\n", \
                    __FILE__, __LINE__, cudaGetErrorString(err)); \
            exit(EXIT_FAILURE); \
        } \
    } while(0)

__global__ void kernel(float *data, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        data[idx] = data[idx] * 2.0f;
    }
}

int main() {
    printf("=== P2P Transfer ===\n\n");

    int n = 1000000;
    size_t size = n * sizeof(float);

    float *h_data = (float*)malloc(size);
    for (int i = 0; i < n; i++) h_data[i] = i;

    float *d_data;
    CUDA_CHECK(cudaMalloc(&d_data, size));
    CUDA_CHECK(cudaMemcpy(d_data, h_data, size, cudaMemcpyHostToDevice));

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    int threads = 256;
    int blocks = (n + threads - 1) / threads;

    cudaEventRecord(start);
    kernel<<<blocks, threads>>>(d_data, n);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);

    float ms;
    cudaEventElapsedTime(&ms, start, stop);

    CUDA_CHECK(cudaMemcpy(h_data, d_data, size, cudaMemcpyDeviceToHost));

    printf("Processed %d elements in %.2f ms\n", n, ms);
    printf("Bandwidth: %.2f GB/s\n", (size * 2 / 1e9) / (ms / 1000.0));

    free(h_data);
    cudaFree(d_data);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);

    return 0;
}

## Key Takeaways

1. Scale across multiple GPUs
2. cudaSetDevice for GPU selection
3. Data parallelism patterns
4. P2P transfers between GPUs

## Next StepsContinue to: **35_next_topic.ipynb**

## Notes*Use this space to write your own notes and observations:*------